<div style="background-color: #ADD8E6; border-radius: 12px; padding: 20px; border: 3px solid #87CEEB;">
    <strong style="color: #000000; font-size: 1.5em;">Modelos de predicción</strong>
</div>

In [3]:
# Importaciones necesarias para el correcto funcionamiento de todos lo modelos y demás operaciones con los datos
import pandas as pd 
import numpy as np
import sklearn
import warnings
import subprocess
import importlib
# warnings.filterwarnings("ignore")

C:\Users\34634\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\34634\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
result_df = pd.read_csv("Fitabase Data 4.12.16-5.12.16/test_train_data.csv")
print(result_df)

                Id                 Time   HeartRate  Intensity  Calories
0       2022484408  2016-04-12 07:21:00  101.600000          1   3.32064
1       2022484408  2016-04-12 07:22:00   87.888889          1   3.94326
2       2022484408  2016-04-12 07:23:00   58.000000          0   1.34901
3       2022484408  2016-04-12 07:24:00   58.000000          0   1.03770
4       2022484408  2016-04-12 07:25:00   56.777778          0   1.03770
...            ...                  ...         ...        ...       ...
333141  8877689391  2016-05-12 13:55:00   60.666667          0   1.33353
333142  8877689391  2016-05-12 13:56:00   61.875000          0   1.33353
333143  8877689391  2016-05-12 13:57:00   58.142857          0   1.33353
333144  8877689391  2016-05-12 13:58:00   61.200000          0   1.33353
333145  8877689391  2016-05-12 13:59:00   58.000000          0   1.33353

[333146 rows x 5 columns]


# Instalación/Actualización de librerías necesarias

In [5]:
from packaging import version

In [6]:
try:
    importlib.import_module('xgboost')
    print("XGBoost ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'xgboost'])

XGBoost ya está instalado en el sistema.


In [7]:
try:
    importlib.import_module('xgboost')
    print("LightGBM ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'lightgbm'])

LightGBM ya está instalado en el sistema.


In [8]:
try:
    importlib.import_module('xgboost')
    print("TensorFlow ya está instalado en el sistema.")
except ImportError:
    subprocess.run(['pip', 'install', 'tensorflow', 'scikit-learn'])

TensorFlow ya está instalado en el sistema.


In [9]:
subprocess.run(['pip', 'install', '--upgrade', 'lightgbm', 'pandas', 'dask'])

CompletedProcess(args=['pip', 'install', '--upgrade', 'lightgbm', 'pandas', 'dask'], returncode=0)

In [10]:
required_version = '14.0.1'

try:
    pyarrow_version = importlib.import_module('pyarrow').__version__
    if version.parse(pyarrow_version) < version.parse(required_version):
        print(f"PyArrow versión {pyarrow_version} encontrada. Actualizando a la versión {required_version}.")
        subprocess.run(['pip', 'install', f'pyarrow>={required_version}'])
    else:
        print(f"PyArrow versión {pyarrow_version} ya está instalada.")
except ImportError:
    subprocess.run(['pip', 'install', f'pyarrow>={required_version}'])


PyArrow versión 15.0.0 ya está instalada.


<div style="background-color: #ADD8E6; border-radius: 12px; padding: 20px; border: 3px solid #87CEEB;">
    <strong style="color: #000000; font-size: 1.5em;">⚠ **User Alert:**</strong> Don't forget to RESTART the kernel for the magic to happen after ADOPTING the new libraries.
</div>

# Preprocesamiento de los datos

In [11]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score
from sklearn.pipeline import Pipeline

# Convertir la columna 'Time' a tipo datetime en data_intensities_minutes
result_df['Time'] = pd.to_datetime(result_df['Time'])

# Extraemos características de la columna 'Time'
result_df['Hour'] = result_df['Time'].dt.hour
result_df['Minutes'] = result_df['Time'].dt.minute
result_df['Weekday'] = result_df['Time'].dt.weekday

# Definir las características y la variable objetivo
features = ['Id', 'Hour', 'Minutes', 'Intensity', 'Calories']
target = 'HeartRate'

# Separar las características y la variable objetivo
X = result_df[features]
y = result_df[target]

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.17, random_state=777)

# Crear un transformador para manejar las variables categóricas y escalar las numéricas
categorical_features = ['Id']
numeric_features = ['Hour', 'Minutes', 'Intensity', 'Calories']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Gradient Boosted Trees : XGBoost

In [12]:
from xgboost import XGBRegressor

num_splits = 5 
xgb_model = XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_scores = cross_val_score(xgb_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPromedio R2 Score (XGBoost): {xgb_scores.mean()}\033[0m")

Promedio R2 Score (XGBoost): 0.858344518126301


In [13]:
from xgboost import XGBRegressor

# Mejores hiperparámetros encontrados previamente utilizado GridSearch
best_params = {'colsample_bytree': 1.0, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}

# Crear el modelo XGBoost con los mejores hiperparámetros
optimal_model = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    n_estimators=best_params['n_estimators'],
    learning_rate=best_params['learning_rate'],
    max_depth=best_params['max_depth'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree']
)

num_splits = 5
xgb_scores = cross_val_score(optimal_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))
print(f"\033[94;1mPromedio R2 Score (XGBoost) con mejores hiperparámetros: {xgb_scores.mean()}\033[0m")

Promedio R2 Score (XGBoost) con mejores hiperparámetros: 0.8604436463886229


# LightGBM

In [14]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score, KFold

# Mejores hiperparámetros encontrados previamente con GridSearch
best_params = {'learning_rate': 0.2, 'n_estimators': 200, 'num_leaves': 40}

best_model = lgb.LGBMRegressor(objective='regression', random_state=42, force_row_wise=True, **best_params)

num_splits = 5 
lgb_scores = cross_val_score(best_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPromedio R2 Score (LightGBM): {lgb_scores.mean()}\033[0m")

C:\Users\34634\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py:150: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] El sistema no puede encontrar el archivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\34634\anaconda3\lib\site-packages\joblib\externals\loky\backend\context.py", line 227, in _count_physical_cores
    cpu_info = subprocess.run(
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\34634\anaconda3\lib\subprocess.py", line 1420, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,


[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266516, number of used features: 5
[LightGBM] [Info] Start training from score 73.702794
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 5
[LightGBM] [Info] Start training from score 73.701315
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 5
[LightGBM] [Info] Start training from score 73.703051
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 5
[LightGBM] [Info] Start training from score 73.734438
[LightGBM] [Info] Total Bins 358
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 5
[LightGBM] [Info] Start training from score 73.707150
Promedio R2 Score (LightGBM): 0.8608125059372875


In [15]:
# Mejores hiperparámetros encontrados previamente con GridSearch
best_params = {'learning_rate': 0.2, 'n_estimators': 200, 'num_leaves': 40}

best_model = lgb.LGBMRegressor(objective='regression', random_state=42, **best_params)

num_splits = 5 
lgb_params = best_params.copy()
lgb_params['force_row_wise'] = True

best_model_force_row_wise = lgb.LGBMRegressor(objective='regression', random_state=42, **lgb_params)
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', best_model_force_row_wise)])
lgb_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"\033[94;1mPromedio R2 Score (LightGBM): {lgb_scores.mean()}\033[0m")

[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266516, number of used features: 18
[LightGBM] [Info] Start training from score 73.702794
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 18
[LightGBM] [Info] Start training from score 73.701315
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 18
[LightGBM] [Info] Start training from score 73.703051
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 18
[LightGBM] [Info] Start training from score 73.734438
[LightGBM] [Info] Total Bins 374
[LightGBM] [Info] Number of data points in the train set: 266517, number of used features: 18
[LightGBM] [Info] Start training from score 73.707150
Promedio R2 Score (LightGBM): 0.8555762087754732


# LSTM: Long Short Term Memory

In [16]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train[numeric_features])
X_test_scaled = scaler.transform(X_test[numeric_features])

# Reshape para el formato de entrada de LSTM (n_samples, time steps, features)
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

# Construir el modelo LSTM
model = Sequential()
model.add(LSTM(120, activation='relu', input_shape=(1, len(numeric_features))))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
model.fit(X_train_reshaped, y_train, epochs=15, batch_size=32, verbose=1)

y_pred = model.predict(X_test_reshaped)

# Calcular el R2 score
r2 = r2_score(y_test, y_pred)
print(f"\033[94;1mR2 Score: {r2:.6f}\033[0m")





Epoch 1/15

8641/8641 [==============================] - 31s 3ms/step - loss: 375.6779
Epoch 2/15
8641/8641 [==============================] - 33s 4ms/step - loss: 120.2599
Epoch 3/15
8641/8641 [==============================] - 30s 3ms/step - loss: 118.7805
Epoch 4/15
8641/8641 [==============================] - 28s 3ms/step - loss: 117.7508
Epoch 5/15
8641/8641 [==============================] - 28s 3ms/step - loss: 116.7752
Epoch 6/15
8641/8641 [==============================] - 32s 4ms/step - loss: 115.7455
Epoch 7/15
8641/8641 [==============================] - 27s 3ms/step - loss: 114.8084
Epoch 8/15
8641/8641 [==============================] - 29s 3ms/step - loss: 114.1138
Epoch 9/15
8641/8641 [==============================] - 28s 3ms/step - loss: 113.5860
Epoch 10/15
8641/8641 [==============================] - 26s 3ms/step - loss: 113.2573
Epoch 11/15
8641/8641 [==============================] - 23s 3ms/step - loss: 112.9817
Epoch 12/15
8641/8641 [=========================

# MLPRegressor: MultiLayer Perceptron Regressor

In [ ]:
from sklearn.neural_network import MLPRegressor

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                             ('regressor', MLPRegressor(random_state=42, max_iter=1000))])
num_splits = 5  

# Validación cruzada 
mlp_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"R2 Scores (MLPRegressor) en {num_splits}-fold cross-validation: {mlp_scores}")
print(f"Promedio R2 Score (MLPRegressor): {mlp_scores.mean()}")

C:\Users\34634\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C:\Users\34634\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', MLPRegressor(random_state=42, max_iter=1000))
])

# Define the parameter grid for grid search
param_grid = {
    'regressor__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
    'regressor__alpha': [0.0001, 0.001, 0.01],
    'regressor__activation': ['relu', 'tanh'],
}

# Define the custom scoring function (R2 score)
r2_scorer = make_scorer(r2_score)

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(pipeline, param_grid, cv=KFold(n_splits=5, shuffle=True, random_state=42), scoring=r2_scorer)
grid_search.fit(X, y)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Create a new MLPRegressor with the best hyperparameters
best_model = MLPRegressor(random_state=42, max_iter=1000, **best_params)

# Perform cross-validation with the best model
num_splits = 5
best_mlp_scores = cross_val_score(best_model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"Best Hyperparameters: {best_params}")
print(f"R2 Scores (MLPRegressor) with Best Hyperparameters in {num_splits}-fold cross-validation: {best_mlp_scores}")
print(f"Average R2 Score (MLPRegressor) with Best Hyperparameters: {best_mlp_scores.mean()}")

# KNN Regressor

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

numeric_features = ['Hour', 'Minutes', 'Intensity', 'Calories']
categorical_features = ['Id']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ], remainder='passthrough') 

model = KNeighborsRegressor(n_neighbors=5, algorithm='brute')  # Puedes probar con 'ball_tree' o 'kd_tree' si persiste el problema

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

num_splits = 5
knn_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=42))

print(f"Promedio R2 Score (KNeighborsRegressor): {knn_scores.mean()}")

In [ ]:
n_neighbors_range = np.arange(1, 21)

param_grid = {'model__n_neighbors': n_neighbors_range} 

model = KNeighborsRegressor()
pipeline = Pipeline([
    ('preprocessor', preprocessor), 
    ('model', model)
])


grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')  # Validación cruzada con 5 folds
grid_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_

# Crear el modelo KNN Regressor con los mejores hiperparámetros
model = KNeighborsRegressor(**best_params)  # Desempaqueta los parámetros

# Entrenar el modelo con los mejores hiperparámetros
model.fit(X_train, y_train)

# Evaluar el modelo en el conjunto de prueba
score = model.score(X_test, y_test)
print(f"\033[94;1mR2 Score (con mejores hiperparámetros): {score:.6f}\033[0m")

# Decision Tree Regressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

dt_regressor = DecisionTreeRegressor(random_state=11101)

# Construct the pipeline with the preprocessor and the model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', dt_regressor)
])

# Cross-validation execution
num_splits = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=num_splits, scoring='neg_mean_squared_error')
mse_scores = -scores  # Convert to positive, as cross_val_score returns the negation of MSE

# Train the model on the training set
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

# Calculate mean squared error on the test set
mse_test = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error on Test Set: {mse_test}')

# Confusion Matrix
print("Confusion Matrix:")
print(np.round(np.array([[mse_test]]), decimals=2))

# Additional information about cross-validation scores
print(f'Mean MSE across {num_splits}-fold Cross-Validation: {mse_scores.mean()}')
print(f'MSE Scores for each fold: {mse_scores}')

# SVM: Support Vector Machine Regressor

In [ ]:
from sklearn.svm import SVR

model = SVR(kernel='rbf', gamma='auto', epsilon=0.1)

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
score = r2_score(y_test, y_pred)
print("R2 score on the test set:", score)

# Cross-validation
num_splits = 5
svm_scores = cross_val_score(model, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=11101))

print(f"R2 Scores (SVR) in {num_splits}-fold cross-validation: {svm_scores}")
print(f"Average R2 Score (SVR) across {num_splits}-fold cross-validation: {svm_scores.mean()}")

# Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=100, random_state=11101)

# Create the pipeline that combines preprocessing and the model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('model', model)
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model on the test set
score = pipeline.score(X_test, y_test)

print(f'R2 Score on Test Set: {score}')

# Cross-validation
num_splits = 5
rf_scores = cross_val_score(pipeline, X, y, scoring='r2', cv=KFold(n_splits=num_splits, shuffle=True, random_state=11101))

print(f"R2 Scores (Random Forest) in {num_splits}-fold cross-validation: {rf_scores}")
print(f"Average R2 Score (Random Forest) across {num_splits}-fold cross-validation: {rf_scores.mean()}")